## Create Offline Areas from Feature Layer

This notebook demonstrates how to create map areas for a webmap based on a polygon feature layer.  The resulting map areas can be downloaded and used in ArcGIS Field Maps.

To use this notebook, you need to update variables at the top of the script, including:
1. `offline_map_item_id` to the item id of the webmap you would like to create areas for
2. `feature_layer_item_id` to the item id of the feature layer to use as input areas, `feature_layer_id` to specify the specific layer and `area_name_attribute` for the attribute of the layer to use to name new map areas
3. `output_title_template`, `output_snippet_template`, `output_tags` and `output_folder` to specify properties for the output map areas 

This notebook assumes that:
1. You are the owner of the webmap
2. Your webmap has a raster tile basemap, like Imagery
3. You are using Enterprise 10.7+ or ArcGIS Online with ArcGIS API for Python v2.3.x

For more information on working with map areas see:
- [Managing offline map areas](https://developers.arcgis.com/python/guide/managing-offline-map-areas/)
- [OfflineMapAreaManager API reference](https://developers.arcgis.com/python/api-reference/arcgis.mapping.toc.html#offlinemapareamanager)


In [ ]:
from arcgis.gis import GIS
from arcgis.mapping import WebMap

#The item id of the webmap that you want to create offline areas for
offline_map_item_id = '07a9a65edb07470e9781a5a493f1d92c'
#The item id of the feature layer to use for the areas you want to create
#If the feature layer has more than 16 features, only the first 16 features will be queried
feature_layer_item_id = '12e3f9dadda048e993d504362cf815b4'
#The id of the layer to use
feature_layer_id = 0
#Field name of the attribute to use to name the areas that are created
area_name_attribute = 'Sextant'

#Properties for the output areas
output_title_template = '{} Area'
output_snippet_template = 'A map that contains parks and trails in the {} sextant of Portland, OR, USA.'
output_tags='test'
output_folder = 'pdx'

#Using built-in user
#For information on different authentication schemes see
#https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/
#and for protecting your credentials see
#https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#protecting-your-credentials
gis = GIS('home')

offline_map_item = gis.content.get(offline_map_item_id)
offline_map = WebMap(offline_map_item)

offline_areas_item = gis.content.get(feature_layer_item_id)
offline_areas = offline_areas_item.layers[feature_layer_id].query(result_record_count=16, return_all_records=False)

#If recreating areas for the same map, could cleanup and remove existing preplanned areas
#for ids in offline_map.offline_areas.list():
#    ids.delete()
    
for offline_area in offline_areas.features:
    area_name = offline_area.attributes[area_name_attribute]

    print('Creating offline map area for ' + area_name)
    
    item_prop = {'title': output_title_template.format(area_name),
                 'snippet': output_snippet_template.format(area_name),
                 'tags': [output_tags]}

    try:        
        map_area = offline_map.offline_areas.create(area=offline_area.geometry,
                                                    item_properties=item_prop,
                                                    folder=output_folder),
                                                    # World scale
                                                    min_scale=147914382,
                                                    # Neighborhood scale
                                                    max_scale=20000

    except:
        print('Failed creating map area for ' + area_name)
